In [1]:
import os
import sys

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from models import new_models
from config import load_data

from ray import tune
from ray.air.integrations.mlflow import MLflowLoggerCallback
from ray.tune.schedulers import ASHAScheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau

import mlflow
from mlflow.tracking import MlflowClient

# Training setup

In [2]:
def fit(net, loss_function, optimizer, data_loader, num_epochs, mode, lr_scheduler, use_amp=False):
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp) # Mixed-precision support for compatible GPUs
    for epoch in range(num_epochs):
        if epoch < num_epochs - 1:
            keys = ["train", "val"]
        else:
            keys = ["train", "val", "test"]
        for key in keys:
            dataset_size = 0
            dataset_loss = 0.0
            if key == "train":
                net.train()
            else:
                net.eval()
            for X_batch, y_batch in tqdm(data_loader[key]):
                X_batch, y_batch = X_batch.to(mode["device"]), y_batch.to(mode["device"])
                with torch.set_grad_enabled(mode=(key=="train")): # Autograd activated only during training
                    with torch.cuda.amp.autocast(enabled=False): # Mixed-precision support for compatible GPUs
                        batch_output = net(X_batch.float())
                        batch_loss = loss_function(batch_output, y_batch)
                    if key == "train":
                        scaler.scale(batch_loss).backward()
                        scaler.step(optimizer) 	
                        scaler.update()
                        optimizer.zero_grad()
                dataset_size += y_batch.shape[0]
                dataset_loss += y_batch.shape[0] * batch_loss.item()

            dataset_loss /= dataset_size

            # Report results to Ray Tune
            if key == "train":
                tune.report(train_loss=dataset_loss)
            elif key == "val":
                # Update learning rate
                lr_scheduler.step(metrics=dataset_loss)
                tune.report(val_loss=dataset_loss)
            else:
                tune.report(test_loss=dataset_loss)
    return net

In [3]:
from config import load_data

def train_model(config, data_dir):

    use_GPU = torch.cuda.is_available()
    if use_GPU:
        mode = {"name": "cuda", "device": torch.device("cuda")}
    else:
        mode = {"name": "cpu", "device": torch.device("cpu")}

    # Define hyperparameters
    train_size = 0.7
    val_size = 0.2
    test_size = 0.1

    sequence_length = config['sequence_length']
    batch_size = config['batch_size']
    num_epochs = config['num_epochs']
    lr = config['lr']
    weight_decay = config['weigth_decay']
    vars = config['variables']

    ld = load_data(data_dir = data_dir, target_variable = config['target_variable'])
    
    X, y = ld.create_lagged_matrix(window_size=sequence_length, vars_to_lag=vars)

    X_train, y_train, X_val, y_val, X_test, y_test = ld.split_data(X, y, train_size=train_size, val_size=val_size, test_size=test_size)

    train_dataloader = ld.create_dataloader(X_train, y_train, sequence_length, batch_size=batch_size, shuffle=True)
    val_dataloader = ld.create_dataloader(X_val, y_val, sequence_length, batch_size=batch_size, shuffle=True)
    test_dataloader = ld.create_dataloader(X_test, y_test, sequence_length, batch_size=batch_size, shuffle=False)
    
    # Model inputs
    if vars:
        input_size = len(vars) + 1
    else:
        input_size = 1
    hidden_size = config['hidden_size']
    num_layers = config['num_layers']
    output_size = 1

    if config['arch'] == "FCN":
        net = new_models.FCN(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )
    elif config['arch'] ==  "FCNTemporalAttention":
        net = new_models.FCNTemporalAttention(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )
    elif config['arch'] == "LSTM":
        net = new_models.LSTM(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )
    elif config['arch'] == "LSTMTemporalAttention":
        net = new_models.LSTMTemporalAttention(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )
    elif config['arch'] == "LSTMSpatialTemporalAttention":
        net = new_models.LSTMSpatialTemporalAttention(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )

    data_loader = {
    "train": train_dataloader,
    "val": val_dataloader,
    "test": test_dataloader,
    }
    
    net.to(mode["device"])

    loss_function = nn.MSELoss().to(mode["device"])
    optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)

    # Define your learning rate scheduler
    lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
                                           
    best_net = fit(net, loss_function, optimizer, data_loader, num_epochs, mode, lr_scheduler, use_amp=True)
    out_name = ""
    for k, v in config.items():
        if not k in ['weights_dir', 'cwd', 'variables']:
            out_name += '{}-{}_'.format(k, v)
    torch.save(best_net.state_dict(), os.path.join(config['cwd'], config['weights_dir'], out_name[:-1] + '.pth'))

# MLFlow setup

In [4]:
client = MlflowClient()
cwd = os.getcwd()
exp_base_name = "Test_of_interface"

created = 0
for i in range(100):
    try:
        exp_name = exp_base_name+"_{}".format(i)
        experiment_id = client.create_experiment(exp_name)
        created=1
        break
    except (TypeError, mlflow.exceptions.MlflowException):
        continue

if not created:
    print("ERROR: Try new experiment name.")
    sys.exit(1)

weights_root = "./model_weights/"
weights_dir = weights_root+exp_name+'/'
os.mkdir(weights_dir)

In [5]:
data_dir = "./data/"
target_variable = 'Q_Kalltveit'

# Start experiments

In [6]:
Discharge = ["Q_Kalltveit_uten_tapping", "Q_Lyngsaana"]
HBV = ["Q_HBV_mean", "Q_HBV", "Evap_HBV", "SNOW_MELT_HBV", "PRECIP_HBV", "GR_WAT_HBV", "TEMP_HBV", "SOIL_WAT_HBV"]
Loggers = ['Vannstand Lyngsåna','Vanntemp. Hiafossen', 'Vannstand Hiafossen', 'Vannstand Kalltveit', 'Vanntemp. Kalltveit kum', 'Vanntemp. Hiavatn', 'Vannstand Hiavatn', 'Vanntemp. Musdalsvatn', 'Vannstand Musdalsvatn', 'Vanntemp. Musdalsvatn nedstrøms', 'Vannstand Musdalsvatn nedstrøms', 'Vanntemp. Viglesdalsvatn', 'Vannstand Viglesdalsvatn', 'Vanntemp. Lyngsåna', 'Vanntemp. Kalltveit elv']
Loggers_1 = ['Vanntemp. Lyngsåna', 'Vannstand Lyngsåna']
Loggers_2 = ['Vanntemp. Kalltveit kum', 'Vannstand Kalltveit']
Loggers_3 = ['Vanntemp. Hiavatn', 'Vannstand Hiavatn']
Loggers_4 = ['Vanntemp. Musdalsvatn', 'Vannstand Musdalsvatn']
Loggers_5 = ['Vanntemp. Musdalsvatn nedstrøms', 'Vannstand Musdalsvatn nedstrøms']
Loggers_6 = ['Vanntemp. Viglesdalsvatn', 'Vannstand Viglesdalsvatn']
Loggers_7 = ['Vanntemp. Kalltveit elv']
Loggers_8 = ['Vannstand Hiafossen']
Meto = ['Nedbør Nilsebu', 'Nedbør Fister', 'Lufttemp Fister', 'Lufttemp. Nilsebu', 'RelHum Nilsebu', 'Vindretning Nilsebu']
Meto_1 = ['Nedbør Nilsebu', 'Lufttemp. Nilsebu', 'RelHum Nilsebu', 'Vindretning Nilsebu']
Meto_2 = ['Nedbør Fister', 'Lufttemp Fister']

Precipitation_1 = ['Nedbør Nilsebu']
Precipitation_2 = ['Nedbør Fister']


In [7]:
from functools import partial

config = {
    "mlflow_experiment_id": experiment_id,
    "weights_dir": weights_dir,
    "cwd": cwd,
    "target_variable": target_variable,
    "arch": tune.grid_search(["LSTMSpatialTemporalAttention"]), # "FCN", "FCNTemporalAttention", "LSTMTemporalAttention", "LSTM", "LSTMSpatialAttention"
    "sequence_length": tune.grid_search([25]),
    'num_epochs': tune.grid_search([150]),
    'num_layers': tune.choice([2, 3, 4]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "weigth_decay": tune.choice([0, 0.001, 0.0001]),
    "batch_size": tune.choice([256, 256*2]),
    "hidden_size": tune.grid_search([64]),
    "variables": tune.grid_search([
        Discharge + HBV + Precipitation_1,
        Discharge + HBV + Precipitation_2,
        Discharge + HBV + Precipitation_1 + Precipitation_2,
    ])
}

analysis = tune.run(
    partial(train_model, data_dir=data_dir),
    config=config,
    resources_per_trial={"cpu": 12, "gpu": 1},
    num_samples=2,
    callbacks=[MLflowLoggerCallback(experiment_name=exp_name)],
)

2023-03-26 14:47:54,863	INFO worker.py:1538 -- Started a local Ray instance.


 84%|████████▍ | 64/76 [00:01<00:00, 94.39it/s]


Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,test_loss,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_model_6e511_00000,2023-03-26_14-50-10,False,,88e6c7a87eec4731947d6a65f5bbf8de,"0_arch=LSTMSpatialTemporalAttention,batch_size=256,hidden_size=64,lr=0.0002,num_epochs=150,num_layers=3,sequence_length=25,variables=Q_Kalltveit_uten_tapping_Q_Lyngsaana_Q_HBV_mean_Q_HBV_Evap_HBV_SNOW_MELT_HBV_PRECIP_HBV_GR_WAT_HBV_TEMP_HBV_SOIL_WAT_HBV_Nedb_r_Nilsebu_Lufttemp_Nilsebu_RelHum_Nilsebu_Vindretning_Nilsebu,weigth_decay=0.0001",DESKTOP-D4IVECG,301,127.0.0.1,3220,9.77203,130.999,0.104644,130.999,1679835010,0,,301,6e511_00000,0.00300431


100%|██████████| 76/76 [00:00<00:00, 116.27it/s]
(func pid=3220) 
100%|██████████| 11/11 [00:00<00:00, 360.06it/s]
2023-03-26 14:50:11,231	ERROR trial_runner.py:1088 -- Trial train_model_6e511_00000: Error processing event.
ray.exceptions.RayTaskError(RuntimeError): ray::ImplicitFunc.train() (pid=3220, ip=127.0.0.1, repr=func)
  File "python\ray\_raylet.pyx", line 830, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 834, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 780, in ray._raylet.execute_task.function_executor
  File "c:\Code\hydro-ml\my_env\lib\site-packages\ray\_private\function_manager.py", line 674, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Code\hydro-ml\my_env\lib\site-packages\ray\util\tracing\tracing_helper.py", line 466, in _resume_span
    return method(self, *_args, **_kwargs)
  File "c:\Code\hydro-ml\my_env\lib\site-packages\ray\tune\trainable\trainable.py", line 367, in train
    raise ski

(func pid=17528) Epoch 00010: reducing learning rate of group 0 to 1.0094e-02.


...

(func pid=17528) Epoch 00139: reducing learning rate of group 0 to 3.9430e-05.


 16%|█▌        | 12/76 [00:00<00:00, 112.32it/s]


(func pid=17528) Epoch 00145: reducing learning rate of group 0 to 1.9715e-05.


100%|██████████| 76/76 [00:00<00:00, 97.10it/s]
(func pid=9280) c:\Code\hydro-ml\my_env\lib\site-packages\pandas\core\indexing.py:1667: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
(func pid=9280)   self.obj[key] = value
2023-03-26 14:53:03,756	ERROR trial_runner.py:1088 -- Trial train_model_6e511_00003: Error processing event.
ray.exceptions.RayTaskError(FileNotFoundError): ray::ImplicitFunc.train() (pid=9280, ip=127.0.0.1, repr=func)
  File "python\ray\_raylet.pyx", line 830, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 834, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 780, in ray._raylet.execute_task.function_executor
  File "c:\Code\hydro-ml\my_env\lib\site-packages\ray\_private\function_manager.py", line 674, i

TuneError: ('Trials did not complete', [train_model_6e511_00000, train_model_6e511_00001, train_model_6e511_00003])